In [5]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
import base64
import io
# from scipy.misc import imread
import codecs
from IPython.display import HTML

movies = pd.read_csv('../dataset/tmdb_5000_movies.csv')
credits = pd.read_csv('../dataset/tmdb_5000_credits.csv')

movies.head(3)
#credits.head(3)

# Possible features: keywords, popularity, production_countries,
# revenue, vote_count


# Clean the dataset by converting json columns into list of strings
fields = ['genres', 'keywords', 'production_companies', 'production_countries']
for field in fields:
    movies[field]=movies[field].apply(json.loads)
    for index,i in zip(movies.index,movies[field]):
        list1=[]
        for j in range(len(i)):
            list1.append((i[j]['name']))
        movies.loc[index,field]=str(list1)
        
        
# Clean the credits 
credits['cast']=credits['cast'].apply(json.loads)
for index,i in zip(credits.index,credits['cast']):
    list1=[]
    for j in range(len(i)):
        list1.append((i[j]['name']))
    credits.loc[index,'cast']=str(list1)

fields = ['crew']
for field in fields:
    credits[field] = credits[field].apply(json.loads)
    def director(x):
        for i in x:
            if i['job'] == 'Director':
                return i['name']
    credits['crew'] = credits['crew'].apply(director)
    credits.rename(columns={'crew':'director'},inplace=True)

In [6]:
# Need to combine two datasets. Can join on movie_id
df = pd.merge(movies, credits, how='left', left_on='id', right_on='movie_id')

In [7]:
# Feature engineering! select which ones we want here...
features = ['id', 'original_title', 'genres', 'cast', 'vote_average', 'director', 'keywords', 'popularity', 'vote_count']
df = df[features]

In [8]:
# Find top genres
df['genres'] = df['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('Adventure', 'Adv').str.replace('u', '').str.replace('Adv', 'Adventure')
df['genres'] = df['genres'].str.split(',')

In [9]:
# Create an ordered list of all possible genres so we can use a bit string to represent them
genreList = []
for genres in df['genres']:
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)
            
genreList.sort()
genreList = genreList[1:]
print(genreList)

['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Docmentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Msic', 'Mystery', 'Romance', 'ScienceFiction', 'TVMovie', 'Thriller', 'War', 'Western']


In [10]:
# Create the bitstring of genres
def genre_bitlist(genres):
    blist = []
    for genre in genreList:
        if genre in genres:
            blist.append(1)
        else:
            blist.append(0)
    return blist

            
df['genres_bs'] = df['genres'].apply(lambda x: genre_bitlist(x))

In [11]:
# Work with cast. Let's get the top 25 actors
n = 25

df['cast']=df['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
df['cast']=df['cast'].str.split(',')


In [12]:
import operator 
actorList = []
list1 = []
for i in df['cast']:
    list1.extend(i)
    
actors = {}
for actor in list1:
    if actor in actors:
        actors[actor] += 1
    else:
        actors[actor] = 1
        
sorted_actors = sorted(actors.items(), key=operator.itemgetter(1))

for i in range(0, 21):
    actorList.append(sorted_actors[-1-i])
    

actorList = [actor[0] for actor in actorList]
alist = []
for actor in actorList:
    if actor != 'r.': # Get rid of this trailing value
        alist.append(actor)
actorList = alist

# Now we need to sort the list!
actorList = sorted(actorList)
actorList = actorList[1:]

In [13]:
# Create the bitstring of genres
def actor_bitlist(actors):
    blist = []
    for actor in actorList:
        if actor in actors:
            blist.append(1)
        else:
            blist.append(0)
    return blist

df['cast_bs'] = df['cast'].apply(lambda x: actor_bitlist(x))

In [14]:
# Now directors
def isnull(s):
    if s is None:
        return ''
    return s

df['director'] = df['director'].apply(isnull)


In [15]:
directorList = []
for director in df['director']:
    if director not in directorList:
        directorList.append(director)
        
directorList = sorted(directorList)[1:]

In [16]:
def director_bitlist(directors):
    blist = []
    for director in directorList:
        if director in directors:
            blist.append(1)
        else:
            blist.append(0)
    return blist

df['director_bs'] = df['director'].apply(lambda x: director_bitlist(x))

In [17]:
# Allows us to make each genre a category in the dataframe
from collections import defaultdict
cols = defaultdict(list)
for g in genreList:
    for index, row in df.iterrows():
        genres = row["genres"]
        if g in genres:
            cols[g].append(1)
        else:
            cols[g].append(0)
        
    df[g] = cols[g]
    

In [18]:
# Allows us to make each actor a category in the dataframe
from collections import defaultdict
cols = defaultdict(list)
for a in actorList:
    for index, row in df.iterrows():
        cast = row["cast"]
        if a in cast:
            cols[a].append(1)
        else:
            cols[a].append(0)
        
    df[a] = cols[a]

In [19]:
nn_features = ['popularity',  'vote_count', 'Action', 'Adventure', 'Animation', 'Comedy', 'Docmentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Mystery', 'Romance', 'ScienceFiction', 'Thriller', 'War', 'Western', 'AlecBaldwin', 'BradPitt', 'BruceWillis', 'JohnGoodman', 'JohnnyDepp', 'LiamNeeson', 'MattDamon', 'MorganFreeman', 'NicolasCage', 'OwenWilson', 'RobertDeNiro', 'SamuelL.Jackson', 'SteveBuscemi', 'WillFerrell', 'WillemDafoe']
target = ['vote_average']

from keras.layers import Input, Dense
from keras.models import Model, Sequential

Using TensorFlow backend.


In [36]:
# print(df.describe())
from sklearn.preprocessing import MinMaxScaler

nn_features = ['popularity',  'vote_count']

X = df[nn_features]
Y = df[target]

X[['popularity','vote_count']] = MinMaxScaler().fit_transform(X[['popularity','vote_count']])
# X[['popularity','vote_count']] = X[['popularity','vote_count']].apply(
#                            lambda x: MinMaxScaler().fit_transform(x))

In [37]:
X

,popularity,vote_count
0,0.171815,0.858057
1,0.158846,0.327225
2,0.122635,0.324753
3,0.128272,0.662158
4,0.050169,0.154450
5,0.132141,0.260035
6,0.055600,0.242147
7,0.153360,0.492074
8,0.112937,0.384889
9,0.177928,0.509308


In [39]:
model = Sequential()
model.add(Dense(2, input_dim=2, kernel_initializer='normal', activation='linear'))
model.add(Dense(10, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, Y, validation_split=0.2, epochs=10, batch_size=10)
print(model.summary())

Train on 3842 samples, validate on 961 samples
Epoch 1/10
3842/3842 [==============================] - 2s 522us/step - loss: 2.0654 - acc: 2.6028e-04 - val_loss: 1.5814 - val_acc: 0.0010
Epoch 2/10
3842/3842 [==============================] - 1s 330us/step - loss: 1.6541 - acc: 2.6028e-04 - val_loss: 1.5340 - val_acc: 0.0010
Epoch 3/10
3842/3842 [==============================] - 1s 332us/step - loss: 1.6379 - acc: 2.6028e-04 - val_loss: 1.5293 - val_acc: 0.0010
Epoch 4/10
3842/3842 [==============================] - 1s 361us/step - loss: 1.6354 - acc: 2.6028e-04 - val_loss: 1.5280 - val_acc: 0.0010
Epoch 5/10
3842/3842 [==============================] - 1s 339us/step - loss: 1.6345 - acc: 2.6028e-04 - val_loss: 1.5274 - val_acc: 0.0010
Epoch 6/10
3842/3842 [==============================] - 1s 257us/step - loss: 1.6342 - acc: 2.6028e-04 - val_loss: 1.5271 - val_acc: 0.0010
Epoch 7/10
3842/3842 [==============================] - 1s 243us/step - loss: 1.6340 - acc: 2.6028e-04 - val_loss